<a href="https://colab.research.google.com/github/GarimaChaubey/Fake-News-Classifier-Using-LSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Classifier Using LSTM

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [ ]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
df=df.dropna()

In [ ]:
df.shape

(18285, 5)

In [ ]:
#Get the independent features
X=df.drop('label', axis=1)

In [ ]:
#Get the dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.17.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [ ]:
voc_size=5000

# Onehot Representation

In [ ]:
 messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
## Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]', ' ', messages['title'].iloc[i])
  review=review.lower()
  review=review.split()

  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review= ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[664, 3735, 4042, 2121, 4132, 1032, 2695, 62, 3972, 585],
 [3796, 3414, 3970, 3770, 3959, 3070, 2493],
 [3159, 311, 2588, 3849],
 [4198, 3011, 4220, 3633, 131, 210],
 [4005, 3959, 1696, 1940, 620, 2624, 3959, 2155, 4593, 2589],
 [381,
  1883,
  922,
  1882,
  4745,
  2884,
  3020,
  1171,
  1613,
  3674,
  3350,
  434,
  4194,
  3837,
  2493],
 [1567, 1710, 1572, 4069, 3118, 4001, 3756, 3640, 4007, 2425, 3076],
 [849, 371, 1889, 4954, 1266, 2744, 2884, 848, 4007, 2425, 3076],
 [4760, 4278, 2144, 4696, 637, 730, 3331, 369, 2884, 2589],
 [4498, 40, 4677, 1414, 2943, 575, 1661, 120],
 [1073, 3742, 4751, 3880, 3503, 387, 4761, 3682, 3929, 2862, 3387],
 [3633, 3577, 4132, 730, 2884, 1266],
 [1424, 3228, 3933, 412, 3426, 4134, 2451, 3422, 4516],
 [4031, 1638, 4609, 2086, 1257, 4157, 3072, 4007, 2425, 3076],
 [4763, 3463, 4240, 513, 4743, 4007, 2425, 3076],
 [3304, 710, 2038, 390, 4373, 3467, 144, 3464, 3146, 1888],
 [3293, 1911, 3414],
 [2400, 3776, 927, 4357, 2884, 4326, 862, 2493],
 [199,

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr[1]

[3796, 3414, 3970, 3770, 3959, 3070, 2493]

# Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...   62 3972  585]
 [   0    0    0 ... 3959 3070 2493]
 [   0    0    0 ...  311 2588 3849]
 ...
 [   0    0    0 ... 4007 2425 3076]
 [   0    0    0 ... 1861 1579 1888]
 [   0    0    0 ... 1016  624 4414]]


In [ ]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 3796, 3414, 3970, 3770, 3959, 3070, 2493], dtype=int32)

In [ ]:
#Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
len(embedded_docs), y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33, random_state=42)


# Model Training

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - accuracy: 0.7933 - loss: 0.4167 - val_accuracy: 0.9064 - val_loss: 0.2115
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9422 - loss: 0.1481 - val_accuracy: 0.9173 - val_loss: 0.1899
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.9662 - loss: 0.0928 - val_accuracy: 0.9162 - val_loss: 0.2204
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9784 - loss: 0.0668 - val_accuracy: 0.9135 - val_loss: 0.2581
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9858 - loss: 0.0422 - val_accuracy: 0.9140 - val_loss: 0.2774
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.9900 - loss: 0.0337 - val_accuracy: 0.9107 - val_loss: 0.3253
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9947 - loss: 0.0176 - val_accuracy: 0.9102 - val_loss: 0.4268
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9959 - loss: 0.0116 - val_a

# Performance Metrics and Accuracy

In [ ]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [ ]:
y_pred=np.where(y_pred>0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

array([[3107,  312],
       [ 267, 2349]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.904059652029826

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      3419
           1       0.88      0.90      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

